# Isogeny Example

## Parameters
### Choose $p = 2^3 \times 3^2 - 1 = 71$ and $q = p^2 = 71^2$ . Construct the field $\mathbb{F}_q$ using $x^2 + 1$ over $\mathbb{F}_p$. Define the elliptic curve $E : y^2 = x^3 + x$, i.e. $(a,b) = (1,0)$

In [7]:
# Define E over Fq
# p = 2^3*3^2 - 1 = 71 and q = p^2 = 71^2
# i in Fq such that i^2 + 1 = 0
p  = 71
Fq.<i> = GF(p^2, modulus = x^2+1)
E = EllipticCurve(Fq, [1, 0])

print "E  =", E
print "#E =", E.cardinality()
print "jE =", E.j_invariant()

E  = Elliptic Curve defined by y^2 = x^3 + x over Finite Field in i of size 71^2
#E = 5184
jE = 24


## Constuct an isogeny of degree 8
### Pick a point $P$ of order $2^3$, and define the isogeny $\phi$ with $\ker \phi = \langle P \rangle$

In [3]:
P = E.random_point()
while(P.order() != 8):
    P = E.random_point()
phi = E.isogeny(P)
E2  = phi.codomain()

print("P             =", P)
print("phi           =", phi)
print("Domain of phi =", phi.domain())
print("Range of phi  =", phi.codomain())
print("phi(P)        =", phi(P))
print("Degree of phi =", phi.degree())

('P             =', (5*i + 26 : 8*i + 5 : 1))
('phi           =', Isogeny of degree 8 from Elliptic Curve defined by y^2 = x^3 + x over Finite Field in i of size 71^2 to Elliptic Curve defined by y^2 = x^3 + 45*i over Finite Field in i of size 71^2)
('Domain of phi =', Elliptic Curve defined by y^2 = x^3 + x over Finite Field in i of size 71^2)
('Range of phi  =', Elliptic Curve defined by y^2 = x^3 + 45*i over Finite Field in i of size 71^2)
('phi(P)        =', (0 : 1 : 0))
('Degree of phi =', 8)


### Check the homomorphism: Does $\phi(Q) + \phi(R) = \phi(Q+R)$ for any points $R, Q$?

In [4]:
Q = E.random_point()
R = E.random_point()

print "Q             =", Q
print "R             =", R
print "Q + R         =", Q + R
print "phi(Q)+phi(R) =", phi(Q) + phi(R)
print "phi(Q + R)    =", phi(Q+R)
print "Check?        :", phi(Q) + phi(R) == phi(Q+R)

Q             = (27*i + 16 : 34*i + 66 : 1)
R             = (33*i + 2 : 27*i + 13 : 1)
Q + R         = (68*i + 53 : 22*i + 44 : 1)
phi(Q)+phi(R) = (10*i + 32 : 54*i + 58 : 1)
phi(Q + R)    = (10*i + 32 : 54*i + 58 : 1)
Check?        : True


### Another check: The elements which generate the same $\langle P \rangle$ should give the same isogeny map.
#### Pick another point $Q \in \langle P \rangle$ such that $\langle Q \rangle = \langle P \rangle$. For e.g., say $Q = 3P$. Actually, any elements $aP$ with $gcd(a, ord(P)) = 1$ generates $\langle P \rangle$ as well. 

In [5]:
Q = 3*P
Pset = Set([i*P for i in range(8)])
Qset = Set([i*Q for i in range(8)])
phi  = E.isogeny(P)
phi2 = E.isogeny(Q)

print "P             =", P, " of order", P.order()
print "Q             =", Q, " of order", Q.order()
print "Is <Q> = <P>  :", Pset == Qset
print "phi           =", phi
print "phi2          =", phi2
print "Is phi = phi2 :", phi == phi2

P             = (5*i + 26 : 8*i + 5 : 1)  of order 8
Q             = (5*i + 45 : 66*i + 63 : 1)  of order 8
Is <Q> = <P>  : True
phi           = Isogeny of degree 8 from Elliptic Curve defined by y^2 = x^3 + x over Finite Field in i of size 71^2 to Elliptic Curve defined by y^2 = x^3 + 45*i over Finite Field in i of size 71^2
phi2          = Isogeny of degree 8 from Elliptic Curve defined by y^2 = x^3 + x over Finite Field in i of size 71^2 to Elliptic Curve defined by y^2 = x^3 + 45*i over Finite Field in i of size 71^2
Is phi = phi2 : True


### Compute a large-degree isogeny: Let $\ell = 2$ and $e = 3$. Compute the isogeny of degree 8 by using the same point $P$ above. 

In [6]:
print "P      =", P, " of order", P.order(), "\n"

################################################################################
# P is a point on the curve E
# l is a small number, and the order of P is l^e
# Returns (phi0, phi1, ..., phi(e-1)) and the final elliptic curve
def isogeny_walk(P,E,l,e):
    phi = []
    Enext = E
    Pnext = P
    for i in range(0,e):
        #final left edge
        Pf = (l^(e-i-1))*Pnext
        #next isogeny
        phi_next = EllipticCurveIsogeny(Enext,Pf)
        Enext = phi_next.codomain()
        Pnext = phi_next(Pnext)
        phi.append(phi_next)
    return (phi,Enext)
################################################################################
# Compute the new isogeny and the final elliptic curve Ef
phi, Ef = isogeny_walk(P,E,2,3)
for i in range(len(phi)):
    print 'phi['+str(i)+'] = ', phi[i]
print ""

# Print intermediate curves
print "E[0]  =", phi[0].domain()
for i in range(len(phi)):
    print 'E['+str(i+1)+']  =', phi[i].codomain()
print ""

# Compare with the original isogeny
phi_org = E.isogeny(P)
print "E_f    =", phi_org.codomain()
print ""

Q = E.random_point()
# Compute phi(Q) = phi[2] o phi[1] o phi[0] (Q)
Qf = Q
for i in range(len(phi)):
    Qf = phi[i](Qf)
print "Q           =", Q
print "phi_org(Q)  =", phi_org(Q)
print "phi_new(Q)  =", Qf

P      = (5*i + 26 : 8*i + 5 : 1)  of order 8 

phi[0] =  Isogeny of degree 2 from Elliptic Curve defined by y^2 = x^3 + x over Finite Field in i of size 71^2 to Elliptic Curve defined by y^2 = x^3 + 11*x + 57*i over Finite Field in i of size 71^2
phi[1] =  Isogeny of degree 2 from Elliptic Curve defined by y^2 = x^3 + 11*x + 57*i over Finite Field in i of size 71^2 to Elliptic Curve defined by y^2 = x^3 + 30*x + 31*i over Finite Field in i of size 71^2
phi[2] =  Isogeny of degree 2 from Elliptic Curve defined by y^2 = x^3 + 30*x + 31*i over Finite Field in i of size 71^2 to Elliptic Curve defined by y^2 = x^3 + 45*i over Finite Field in i of size 71^2

E[0]  = Elliptic Curve defined by y^2 = x^3 + x over Finite Field in i of size 71^2
E[1]  = Elliptic Curve defined by y^2 = x^3 + 11*x + 57*i over Finite Field in i of size 71^2
E[2]  = Elliptic Curve defined by y^2 = x^3 + 30*x + 31*i over Finite Field in i of size 71^2
E[3]  = Elliptic Curve defined by y^2 = x^3 + 45*i over Finite Fie